# 02. Algebraic Diagnosis and Stability Analysis

**Objective:** Analyze the algebraic properties of the Design Matrix ($X$) to detect multicollinearity and assess numerical stability.

### Key Concepts
1.  **Matrix Rank:** Determines the number of linearly independent features. If $Rank(X) < Columns$, the system has infinite solutions (Perfect Multicollinearity).
2.  **Singular Value Decomposition (SVD):** Decomposes the matrix to reveal its underlying structure and redundant dimensions.
3.  **Condition Number ($\kappa$):** Quantifies the sensitivity of the system to input noise. A high $\kappa$ indicates an "ill-conditioned" problem where matrix inversion is unstable.

In [ ]:
import os
if not os.path.exists('Credit-Risk-Algebraic-ML'):
    !git clone https://github.com/adriangonz-afk/Credit-Risk-Algebraic-ML.git
os.chdir('Credit-Risk-Algebraic-ML')
print(f'✅ Entorno listo en: {os.getcwd()}')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
X = np.load('data/processed/X_matrix.npy')
X_bias = np.c_[np.ones((X.shape[0], 1)), X]
rank = np.linalg.matrix_rank(X_bias)
print(f'Rank: {rank}')
U, S, Vt = np.linalg.svd(X_bias, full_matrices=False)
print(f'Condition Number: {S.max()/S.min():.2e}')
plt.plot(S, 'o-'); plt.yscale('log'); plt.show()